In [ ]:
!pip install geopandas
!pip install folium
!pip install leafmap
!pip install rasterio

In [ ]:
import geopandas as gpd
import folium
import leafmap as leaf
import rasterio
import pandas as pd
import json
import shapely
import requests
import matplotlib.pyplot as plt
import numpy as np
from pyproj import Transformer
from rasterio.mask import mask
import datetime
from shapely.geometry import Point
import fiona

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Ερώτημα 1**

In [ ]:
study_area = gpd.read_file('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/map.geojson')

In [ ]:
print(study_area.head())

In [ ]:
kastoria = rasterio.open('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/Kastoria/Kastoria.tif')

In [ ]:
type(kastoria)

In [ ]:
kastoria.meta

WGS 84 / UTM zone 34N

In [ ]:
band1 = kastoria.read(1)
print("\nRaster Data (Band 1):")
print(band1)

In [ ]:
nasa=pd.read_csv('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/Kastoria/POWER_Point_Daily_20160101_20161231_040d5404N_021d3403E_LST_mdf.csv')

In [ ]:
nasa

In [ ]:
dates = [
    (2016, 1, 25),
    (2016, 2, 14),
    (2016, 3, 28),
    (2016, 4, 4),
    (2016, 4, 27),
    (2016, 6, 6),
    (2016, 7, 3),
    (2016, 7, 13),
    (2016, 7, 23),
    (2016, 7, 26),
    (2016, 8, 5),
    (2016, 8, 12),
    (2016, 8, 15),
    (2016, 8, 22),
    (2016, 9, 4),
    (2016, 9, 14),
    (2016, 10, 1),
    (2016, 10, 14),
    (2016, 11, 13),
    (2016, 12, 3),
    (2016, 12, 10),
    (2016, 12, 20),
    (2016, 12, 23),
    (2016, 12, 30),
]
print(dates)

In [ ]:
dates_df = pd.DataFrame(dates, columns=['YEAR', 'MO', 'DY'])

In [ ]:
dates_df = pd.DataFrame(dates, columns=['YEAR', 'MO', 'DY'])
nasa = pd.merge(nasa, dates_df, on=['YEAR', 'MO', 'DY'])

In [ ]:
nasa

In [ ]:
def _wkt2esri(wkt:str)->str:
    """Converts WKT geometries to arcGIS geometry strings.

    Args:
        wkt (str): WKT geometry string
    Returns:
        str: ESRI arcGIS polygon geometry string
    """
    geom = shapely.wkt.loads(wkt)
    rings = None
    # Testing for polygon type
    if geom.geom_type == 'MultiPolygon':
        rings = []
        for pg in geom.geoms:
            rings += [list(pg.exterior.coords)] + [list(interior.coords) for interior in pg.interiors]
    elif geom.geom_type == 'Polygon':
        rings = [list(geom.exterior.coords)] + [list(interior.coords) for interior in geom.interiors]
    else:
        print("Shape is not a polygon")
        return None

    # Convert to esri geometry json
    esri = json.dumps({'rings': rings})

    return esri

def corine(aoi:str, to_file:bool = False, fname:str = "corine_2018.shp")->tuple:
    """Downloads Corine Land Cover 2018 data from Copernicus REST API.

    Args:
        aoi (str): Path to file with the region of interest
        to_file (bool, optional): Save result to file. Defaults to False
        fname (str, optional): Path and name of the created file. Defaults to "corine_2018.shp"
    Returns:
        tuple: Corine Land Cover 2018 data as GeoDataFrame and the path to saved file
    """
    HTTP_OK = 200

    geoms = gpd.read_file(aoi).dissolve()
    polygons = list(geoms.geometry)
    wkt = f"{polygons[0]}"
    esri = _wkt2esri(wkt)
    # Build URL for retrieving data
    server = "https://image.discomap.eea.europa.eu/arcgis/rest/services/Corine/CLC2018_WM/MapServer/0/query?"
    payload = {
        "geometry": esri,
        "f": "GeoJSON",
        "inSR": geoms.crs.to_epsg(),
        "geometryType": "esriGeometryPolygon",
        "spatialRel": "esriSpatialRelIntersects",
        "returnGeometry": True
        }
    print ("Starting retrieval...")
    request = requests.get(server, params = payload)
    # Check if server didn't respond to HTTP code = 200
    if request.status_code != HTTP_OK:
        raise requests.exceptions.HTTPError("Failed retrieving POWER data, server returned HTTP code: {} on following URL {}.".format(request.status_code, request.url))
    # In other case is successful
    print ("Successfully retrieved data!")
    json_data = request.json()
    data = gpd.GeoDataFrame.from_features(json_data)
    if to_file:
        data.to_file(fname)

    return data, fname

In [ ]:
study_area_input, study_area_shp = corine('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/map.geojson', True, '/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/study_area.shp')

**Ερώτημα Β**

Θα οπτικοποιήσω τα κανάλια Blue, Green και Red από το "kastoria" για την ημερομηνία 26/07/2016. Άρα, τα 91ο, 92ο και 93ο κανάλια αντίστοιχα.

In [ ]:
blue = kastoria.read(91)
green = kastoria.read(92)
red = kastoria.read(93)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(blue, cmap='Blues')
axes[0].set_title('Blue Channel')

axes[1].imshow(green, cmap='Greens')
axes[1].set_title('Green Channel')

axes[2].imshow(red, cmap='Reds')
axes[2].set_title('Red Channel')

plt.tight_layout()
plt.savefig('B_G_R.png')
plt.show()

In [ ]:
rgb_image = np.stack([red, green, blue], axis=-1)

plt.figure(figsize=(8, 8))
plt.imshow(rgb_image)
plt.title('RGB Image')
plt.axis('off')
plt.savefig('RGB.png')
plt.show()

Ελέγχω το σύστημα συντεταγμένων για το study_area shapefile.

Με χρήση qgis βλέπω για το study_area EPSG:4326 (εδώ έδινε None).

In [ ]:
source_crs = 'EPSG:4326'  # WGS 84
target_crs = 'EPSG:32634'  # WGS 84 / UTM zone 34N

transformer = Transformer.from_crs(source_crs, target_crs)

study_area_reprojected = study_area.to_crs(target_crs)

study_area_reprojected.to_file('study_area_reprojected.shp')

In [ ]:
study_area_reprojected.geometry.values[0]

In [ ]:
study_area_geom = study_area_reprojected.geometry.values[0]
print(study_area_geom)

In [ ]:
clipped_image, clipped_transform=mask(kastoria, [study_area_geom], crop=True)

In [ ]:
clipped_meta = kastoria.meta.copy()
clipped_meta.update({
    'height': clipped_image.shape[1],
    'width': clipped_image.shape[2],
    'transform': clipped_transform
})

In [ ]:
clipped_meta

In [ ]:
# with rasterio.open('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/clipped_image.tif', 'w', **clipped_meta) as clp:
#   clp.write(clipped_image)

In [ ]:
with rasterio.open('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/clipped_image.tif') as clp:

  red_cl = clp.read(93)
  green_cl = clp.read(92)
  blue_cl = clp.read(91)

  clipped_rgb_image = np.stack([red_cl, green_cl, blue_cl], axis=-1)

  fig, ax = plt.subplots(figsize=(10, 10))

  ax.imshow(clipped_rgb_image)
  ax.set_title('Study Area - RGB', fontsize=15)
  plt.savefig('clipped_kastoria.png')
  plt.show()

In [ ]:
NDVI_bands = list(range(7, 238, 10))
print(NDVI_bands)

In [ ]:
NDWI_bands = list(range(9, 240, 10))
print(NDWI_bands)

In [ ]:
NDBI_bands = list(range(10, 241, 10))
print(NDBI_bands)

In [ ]:
NDVIs = []
NDWIs = []
NDBIs = []

with rasterio.open('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/clipped_image.tif') as clp:

    for i in NDVI_bands:
      NDVI = clp.read(i)
      NDVIs.append(NDVI)

    for j in NDWI_bands:
      NDWI = clp.read(j)
      NDWIs.append(NDWI)

    for k in NDBI_bands:
     NDBI = clp.read(k)
     NDBIs.append(NDBI)

In [ ]:
stacked_ndvis = np.stack(NDVIs, axis=0)
mean_NDVI = np.mean(stacked_ndvis, axis=0)
print("Mean NDVI:", mean_NDVI)

In [ ]:
stacked_ndwis = np.stack(NDWIs, axis=0)
mean_NDWI = np.mean(stacked_ndwis, axis=0)
print("Mean NDWI:", mean_NDWI)

In [ ]:
stacked_ndbis = np.stack(NDBIs, axis=0)
mean_NDBI = np.mean(stacked_ndbis, axis=0)
print("Mean NDBI:", mean_NDBI)

In [ ]:
for i, ndvi_values in enumerate(NDVIs):

    plt.figure(figsize=(10, 8))
    plt.imshow(ndvi_values, cmap='viridis')
    plt.colorbar(label='NDVI')
    plt.title(f'NDVI for Date {i+1}')
    plt.xlabel('Pixel Column')
    plt.ylabel('Pixel Row')
    plt.show()

In [ ]:
for i, ndwi_values in enumerate(NDWIs):

    plt.figure(figsize=(10, 8))
    plt.imshow(ndwi_values, cmap='Blues')
    plt.colorbar(label='NDWI')
    plt.title(f'NDWI for Date {i+1}')
    plt.xlabel('Pixel Column')
    plt.ylabel('Pixel Row')
    plt.show()

In [ ]:
for i, ndbi_values in enumerate(NDBIs):

    plt.figure(figsize=(10, 8))
    plt.imshow(ndvi_values, cmap='cividis')
    plt.colorbar(label='NDBI')
    plt.title(f'NDBI for Date {i+1}')
    plt.xlabel('Pixel Column')
    plt.ylabel('Pixel Row')
    plt.show()

In [ ]:
datetimes = [datetime.date(year, month, day) for year, month, day in dates]

mean_ndvi_values = [np.mean(ndvi) for ndvi in NDVIs]
mean_ndwi_values = [np.mean(ndwi) for ndwi in NDWIs]
mean_ndbi_values = [np.mean(ndbi) for ndbi in NDBIs]

plt.figure(figsize=(10, 6))
plt.plot(datetimes, mean_ndvi_values, color='g', linestyle='-', label = "NDVI")
plt.plot(datetimes, mean_ndwi_values, color='b', linestyle='-', label = "NDWI")
plt.plot(datetimes, mean_ndbi_values, color='r', linestyle='-', label = "NDBI")
plt.title('Indices Over Time')
plt.xlabel('Date')
plt.ylabel('Mean Value')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('indices_over_time.png')
plt.show()

**Ερώτημα 3**

Διαβάζουμε τα δεδομένα (υπό μορφή GeoJSON και στο σύστημα αναφοράς WGS84) για αεροδρόμια, φράγματα, ποτάμια,
ακτογραμμή, όρια περιφερειακών ενοτήτων και περιφερειών.

In [ ]:
regions = gpd.read_file('https://geodata.gov.gr/geoserver/wfs/?service=WFS&version=1.0.0&request=GetFeature&typeName=geodata.gov.gr:d7f50467-e5ef-49ac-a7ce-15df3e2ed738&outputFormat=application/json&srsName=epsg:4326')
reg_units = gpd.read_file('https://geodata.gov.gr/geoserver/wfs/?service=WFS&version=1.0.0&request=GetFeature&typeName=geodata.gov.gr:0d8c1236-b1dc-4823-85ef-35de6feb07cc&outputFormat=application/json&srsName=epsg:4326')
dams = gpd.read_file('https://geodata.gov.gr/geoserver/wfs/?service=WFS&version=1.0.0&request=GetFeature&typeName=geodata.gov.gr:a16f80c0-862f-42d6-9f5b-92d198030104&outputFormat=application/json&srsName=epsg:4326')
coast = gpd.read_file('https://geodata.gov.gr/geoserver/wfs/?service=WFS&version=1.0.0&request=GetFeature&typeName=geodata.gov.gr:326b7fce-18a8-4248-bdf2-6df8b01b3554&outputFormat=application/json&srsName=epsg:4326')
airports = gpd.read_file('https://geodata.gov.gr/geoserver/wfs/?service=WFS&version=1.0.0&request=GetFeature&typeName=geodata.gov.gr:4f097ff9-4fbb-4411-86f3-1e7e621df61a&outputFormat=application/json&srsName=epsg:4326')
rivers=gpd.read_file('https://geodata.gov.gr/geoserver/wfs/?service=WFS&version=1.0.0&request=GetFeature&typeName=geodata.gov.gr:ece02bd9-8a76-41b1-9e61-093fc63bd944&outputFormat=application/json&srsName=epsg:4326')

In [ ]:
regions

In [ ]:
reg_units

In [ ]:
dams

In [ ]:
coast

In [ ]:
airports

In [ ]:
rivers

Πραγματοποιώ κάποια ερωτήματα, όπως ζητείται. Αρχικά θα βρω τα αεροδρόμια που βρίσκονται στην Περιφέρεια Αν. Μακεδονίας - Θράκης.

In [ ]:
target_region1 = regions[regions['PER'] == 'Π. ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ - ΘΡΑΚΗΣ']
airports_within_region1 = gpd.sjoin(airports, target_region1, op='within')

In [ ]:
airports_within_region1

Τώρα θα βρω το φράγμα που βρίσκεται πιο κοντά στον 1111ο ποταμό του dataframe rivers.

In [ ]:
river1=rivers.iloc[1112]['name']
print(river1)

In [ ]:
target_river = rivers[rivers['name'] == 'ΛΥΓΚΟΣ Π.'].geometry.iloc[0]
dams['distance_to_river'] = dams.geometry.distance(target_river)
nearest_dam = dams.loc[dams['distance_to_river'].idxmin()]

print(nearest_dam)

Τέλος, θα πάρω τα ποτάμια που διασχίζουν την Περιφέρεια Στερεάς Ελλάδας.

In [ ]:
sterea_elladas_region = regions[regions['PER'] == 'Π. ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ']
intersecting_rivers = gpd.sjoin(rivers, sterea_elladas_region, how="inner", op="intersects")

In [ ]:
intersecting_rivers

Ακολούθως απεικονίζεται η περιοχή της Καστοριάς από το Corine Land Cover 2018, μαζί με την περιοχή μελέτης (πολύγωνο) και τη θέση (σημείο) του "μετεωρολογικού σταθμού" που επιλέχθηκε.

In [ ]:
kastoria_polygon = gpd.read_file('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/kastoria.geojson')
foteini = gpd.read_file('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/foteini.geojson')

In [ ]:
kastoria_polygon_input, kastoria_polygon_shp = corine('/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/kastoria.geojson', True, '/content/drive/My Drive/Geospatial Data Analysis/Exercises/Ex1/kastoria_polygon.shp')

In [ ]:
kastoria_polygon_shp=gpd.read_file('kastoria_polygon.shp')

In [ ]:
print(kastoria_polygon_shp.crs)

In [ ]:
kastoria_polygon_shp.set_crs(epsg=4326, inplace=True)

In [ ]:
source_crs = 'EPSG:4326'  # WGS 84
target_crs = 'EPSG:32634'  # WGS 84 / UTM zone 34N

transformer = Transformer.from_crs(source_crs, target_crs)

kastoria_polygon_reprojected = kastoria_polygon_shp.to_crs(target_crs)

kastoria_polygon_reprojected.to_file('kastoria_polygon_reprojected.shp')

In [ ]:
kastoria_polygon_reprojected

In [ ]:
def to_hex2(s: str) -> str:
    return hex(int(s))[2:].zfill(2)

clc_legend = pd.read_csv('clc_legend.csv')
clc_legend = clc_legend.dropna(subset=['RGB'])
clc_legend['RGB_HEX'] = clc_legend['RGB'].str.split('-').map(lambda lst: '#' + ''.join(map(to_hex2, lst)))
clc_legend['CLC_CODE'] = clc_legend['CLC_CODE'].astype(int).astype(str)

code_to_color = dict(zip(clc_legend['CLC_CODE'], clc_legend['RGB_HEX']))

kastoria_polygon_reprojected['color'] = kastoria_polygon_reprojected['Code_18'].map(code_to_color.get)

In [ ]:
foteini

In [ ]:
source_crs = 'EPSG:4326'  # WGS 84
target_crs = 'EPSG:32634'  # WGS 84 / UTM zone 34N

transformer = Transformer.from_crs(source_crs, target_crs)

foteini_reprojected = foteini.to_crs(target_crs)

In [ ]:
ax = kastoria_polygon_reprojected.plot(color=kastoria_polygon_reprojected['color'])
study_area_reprojected.plot(ax=ax)
foteini_reprojected.plot(ax=ax, color='red')
ax.set_axis_off()
ax.set_title('Corine Land Cover 2018')

plt.savefig('corine_land_cover_map.png')

plt.show()

Διαγράμματα θερμοκρασίας, υγρασίας και βροχόπτωσης για τη θέση (Φωτεινή) που επιλέχθηκε εντός του πολυγώνου μελέτης. Τα δεδομένα έχουν αποθηκευτεί στο dataframe 'nasa' από το πρώτο ερώτημα.

In [ ]:
datetimes = [datetime.date(year, month, day) for year, month, day in dates]

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

axs[0].plot(datetimes, nasa['T2M'], color='orange')
axs[0].set_title('2-Meter Temperature (T2M)')
axs[0].set_ylabel('Temperature (°C)')

axs[1].plot(datetimes, nasa['QV2M'], color='green')
axs[1].set_title('Specific Humidity at 2 Meters (QV2M)')
axs[1].set_ylabel('Specific Humidity (kg/kg)')

axs[2].plot(datetimes, nasa['PRECTOTCORR'], color='blue')
axs[2].set_title('Precipitation (PRECTOTCORR)')
axs[2].set_ylabel('Precipitation (mm)')

plt.xlabel('Date')
plt.tight_layout()

plt.savefig('Weather_data.png')

plt.show()

**Ερώτημα 4**

In [ ]:
url = 'https://image.discomap.eea.europa.eu/arcgis/services/Corine/CLC2018_WM/MapServer/WMSServer?request=GetCapabilities&service=WMS'
corine_layer = '12'

m = leaf.Map()
m.add_wms_layer(
    url=url, layers='12', name='Corine Land Cover', format='image/png', shown=True
)

m.add_shp('airports.shp', 'Greece Airports')
m.add_shp('dams.shp', 'Greece Dams')
m.add_shp('regions.shp', 'Regions', style={"color": "black", "weight": 5})
m.add_shp('nomoi.shp', 'Regional Units', style={"color": "red", "weight": 2})

legend_dict = {
    'Greece Airports': 'blue',
    'Greece Dams': 'green',
    'Regions': 'black',
    'Regional Units': 'red'
}

m.add_legend(legend_title="Legend", legend_dict=legend_dict)

In [ ]:
m